# Validating Cowbird user workspace on JupyterLab

This notebook's purpose is to start a JupyterLab Docker container using Jupyterhub in order to validate the user's workspace in a JupyterLab environment that also uses Cowbird.
The JupyterLab container will run the related notebook `resources/user_test_cowbird_jupyter.ipynb` which validates the different directories/files and user permissions.

Also see the note about the usage of the [jhub-client](https://github.com/Quansight/jhub-client) in the notebook `resources/user_test_cowbird_jupyter.ipynb`.

The different test files and the test user are all created in the optional-component [test-cowbird-jupyter-access](https://github.com/bird-house/birdhouse-deploy/tree/master/birdhouse/optional-components/test-cowbird-jupyter-access) on birdhouse. A script is executed by the container when the birdhouse stack is started, which setups everything required for this test. If receiving errors from this test, it could be worth also verifying the container`s logs in case the errors come from the initial setup from birdhouse.

In [1]:
from IPython.utils import io

with io.capture_output() as captured:
    !pip install jhub-client

In [2]:
# define some useful variables for following steps
import json
import os
import requests
import urllib3

VERIFY_SSL = True if 'DISABLE_VERIFY_SSL' not in os.environ else False
if not VERIFY_SSL:
    urllib3.disable_warnings()  # disable warnings for using https without certificate verification enabled

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")
assert PAVICS_HOST != "", "Invalid PAVICS HOST value."

COWBIRD_URL = f"https://{PAVICS_HOST}/cowbird"
JUPYTER_URL = f"https://{PAVICS_HOST}/jupyter/"
JUPYTERHUB_URL = f"https://{PAVICS_HOST}/jupyter/hub"
MAGPIE_URL = f"https://{PAVICS_HOST}/magpie"

def get_credentials(var_name):
    value = os.getenv(var_name)
    if not value:
        raise ValueError("Missing test admin credentials `{}` to run tests.".format(var_name))
    return value

TEST_MAGPIE_ADMIN_USERNAME = get_credentials("TEST_MAGPIE_ADMIN_USERNAME")
TEST_MAGPIE_ADMIN_PASSWORD = get_credentials("TEST_MAGPIE_ADMIN_PASSWORD")

TEST_USER = os.getenv("TEST_COWBIRD_JUPYTERHUB_USERNAME", "testcowbirdjupyter")

In [3]:
def response_msg(message, response, is_json=True):
    """Append useful response details to provided message."""
    _body = response.text
    _detail = "<unknown>"
    if is_json:
        try:
            _body = response.json()
            _detail = _body.get("detail", _body.get("message", "<unknown>"))
        except json.JSONDecodeError:
            # ignore and revert to text body since it could not be parsed as JSON
            _body = response.text
    return "{} Response replied with ({}) [{}]\nContent: {}\n\n".format(message, response.status_code, _detail, _body)

In [4]:
# Check if Cowbird is rolling, else display error
assert requests.get(COWBIRD_URL, verify=VERIFY_SSL).status_code == 200, "Cowbird is not available, but should be actiavted for this test."

In [5]:
def magpie_signin(user_name, password):
    signin_url = f"{MAGPIE_URL}/signin"
    data = {"user_name": user_name, "password": password}
    try:
        resp = requests.request(url=signin_url, headers=HEADERS, method="POST", json=data, timeout=10, verify=VERIFY_SSL)
    except Exception as exc:
        raise RuntimeError(f"Failed to sign in to Magpie (url: `{signin_url}`) with user `{data['user_name']}`. "
                           f"Exception : {exc}. ")
    if resp.status_code != 200:
        raise RuntimeError(f"Unexpected response while trying to sign in to Magpie with user `{user_name}` : {resp.text}")
    return resp

In [6]:
magpie_admin_session = requests.Session()
magpie_admin_session.verify = VERIFY_SSL
magpie_admin_session.headers = HEADERS
magpie_admin_session.cookies = magpie_signin(TEST_MAGPIE_ADMIN_USERNAME, TEST_MAGPIE_ADMIN_PASSWORD).cookies

In [7]:
# Make sure user exists on JupyterHub
resp = magpie_admin_session.get(f"{JUPYTERHUB_URL}/api/users",
    headers={"Authorization": "Token admin-token", "Accept": "application/json"})
usernames = [user["name"] for user in resp.json()]

if TEST_USER not in usernames:
    resp = magpie_admin_session.post(f"{JUPYTERHUB_URL}/api/users/{TEST_USER}",
        headers={"Authorization": "Token admin-token", "Accept": "application/json"})
    if resp.status_code != 201:
        raise ValueError(response_msg("\nFailed to create JupyterHub test user.", resp))

In [8]:
# Get JupyterLab user token
resp = magpie_admin_session.post(
    f"{JUPYTERHUB_URL}/api/users/{TEST_USER}/tokens", 
    headers={"Authorization": "Token admin-token", "Accept": "application/json"})
if resp.status_code != 201:
    raise ValueError(response_msg("\nFailed to create JupyterLab user token.", resp))

# Set env variable required by the jhub-client package
os.environ["JUPYTERHUB_API_TOKEN"] = resp.json()["token"]

In [9]:
if VERIFY_SSL:
    !jhubctl run --hub {JUPYTER_URL} --auth-type token -u {TEST_USER} -n resources/user_test_cowbird_jupyter.ipynb \
      --output-filename notebook_results.txt --stop-server --validate
else:
    !jhubctl run --hub {JUPYTER_URL} --auth-type token -u {TEST_USER} -n resources/user_test_cowbird_jupyter.ipynb \
      --no-verify-ssl --output-filename notebook_results.txt --stop-server --validate

INFO:jhub_client.api:creating cluster username=testcowbirdjupyter user_options={}
INFO:jhub_client.api:created server for username=testcowbirdjupyter with user_options={}
INFO:jhub_client.api:created kernel_spec=python3 kernel=126219f0-6ad7-4c24-b565-efd57fa05851 for jupyter
INFO:jhub_client.api:deleted kernel=126219f0-6ad7-4c24-b565-efd57fa05851 for jupyter
INFO:jhub_client.api:deleted server for username=testcowbirdjupyter
